In [2]:
# https://medium.com/@tomar.ankur287/user-user-collaborative-filtering-recommender-system-51f568489727

import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')
Ratings = pd.read_csv('data/parsed_combined_data_1.txt')

In [3]:
Ratings.columns = ['movieId', 'userId', 'rating', 'timestamp']

In [4]:
Ratings

,movieId,userId,rating,timestamp
0,1,822109,5,2005-05-13
1,1,885013,4,2005-10-19
2,1,30878,4,2005-12-26
3,1,823519,3,2004-05-03
4,1,893988,3,2005-11-17
...,...,...,...,...
24053758,4499,2591364,2,2005-02-16
24053759,4499,1791000,2,2005-02-10
24053760,4499,512536,5,2005-07-27
24053761,4499,988963,3,2005-12-20


In [5]:
Mean= Ratings.groupby(['userId'], as_index = False, sort = False).mean().rename(columns = {'rating': 'rating_mean'})[['userId','rating_mean']]
Ratings = pd.merge(Ratings,Mean,on = 'userId', how = 'left', sort = False)
Ratings['rating_adjusted']=Ratings['rating']-Ratings['rating_mean']
Ratings

,movieId,userId,rating,timestamp,rating_mean,rating_adjusted
0,1,822109,5,2005-05-13,4.083333,0.916667
1,1,885013,4,2005-10-19,3.873563,0.126437
2,1,30878,4,2005-12-26,3.634304,0.365696
3,1,823519,3,2004-05-03,3.917197,-0.917197
4,1,893988,3,2005-11-17,3.190141,-0.190141
...,...,...,...,...,...,...
24053758,4499,2591364,2,2005-02-16,3.067729,-1.067729
24053759,4499,1791000,2,2005-02-10,3.972973,-1.972973
24053760,4499,512536,5,2005-07-27,4.085366,0.914634
24053761,4499,988963,3,2005-12-20,3.594340,-0.594340


In [6]:
distinct_movie = np.unique(Ratings['movieId'])
distinct_users = np.unique(Ratings['userId'])

In [36]:
single_user_data = Ratings.groupby(['userId'], as_index = False, sort = False).agg({'rating': 'mean', 'rating_adjusted': lambda x: np.sqrt(np.sum(np.square(x), axis = 0))})

single_user_data.rename(columns={'rating_adjusted': 'vals'}, inplace=True)

single_user_data = Ratings.loc[:, ['movieId', 'userId', 'rating_adjusted']].join(single_user_data.set_index('userId'), on = 'userId')

single_user_data.rename(columns={'userId': 'userId1', 'rating_adjusted': 'rating_adjusted1'}, inplace=True)



In [37]:
single_user_data

,movieId,userId1,rating_adjusted1,rating,vals
0,1,822109,0.916667,4.083333,4.769696
1,1,885013,0.126437,3.873563,9.033781
2,1,30878,0.365696,3.634304,14.130689
3,1,823519,-0.917197,3.917197,10.484444
4,1,893988,-0.190141,3.190141,9.688457
...,...,...,...,...,...
24053758,4499,2591364,-1.067729,3.067729,13.922952
24053759,4499,1791000,-1.972973,3.972973,6.853683
24053760,4499,512536,0.914634,4.085366,8.148769
24053761,4499,988963,-0.594340,3.594340,10.370950


In [42]:
def get_movie_rating(movie_id, user_id):
    user_data_append=pd.DataFrame()

    user1_data =  single_user_data.loc[single_user_data['userId1'] == user_id, :]
    
    movie = movie_id

    item_user =  Ratings[Ratings['movieId']==movie]

    distinct_users1 = np.unique(item_user['userId'])

    if len(distinct_users1) == 0:
        return pd.DataFrame([0])
    j = 1
    
    # print('distinct_users1: ' + str(len(distinct_users1)))

    # filter ratings for distinct users
    ratings_distinct_users = Ratings[Ratings['userId'].isin(distinct_users1)] 
    
    rms_vals = ratings_distinct_users.groupby('userId').apply(lambda x: np.sqrt(np.sum(np.square(x['rating_adjusted']), axis=0)))
    rms_vals.name = 'rms_vals'
    
    ratings_distinct_users = ratings_distinct_users.merge(rms_vals, left_on='userId', right_index=True, how='left')
    
    ratings_distinct_users.rename(columns={'rating_adjusted':'rating_adjusted2'}, inplace=True)
    ratings_distinct_users.rename(columns={'userId':'userId2'}, inplace=True)
    
    user_data = pd.merge(user1_data, ratings_distinct_users,on='movieId',  how = 'inner', sort = False)
    
    user_data['dot'] = (user_data['rating_adjusted1']*user_data['rating_adjusted2'])/(user1_data['vals']*user_data['rms_vals'])

    # print('user comparison done')
    user_data = user_data[user_data['dot']<1].nlargest(30, 'dot')
    user_data['movieId'] = movie
    user_data_append = pd.concat([user_data_append, user_data], ignore_index=True)
 
    
    User_dot_adj_rating_all = pd.DataFrame()

    user_data_append_movie = user_data_append[user_data_append['movieId']==movie]
    User_dot_adj_rating = pd.merge(Ratings,user_data_append_movie[['dot','userId2','userId1']], how = 'inner',left_on='userId', right_on='userId2', sort = False)

    User_dot_adj_rating1 = User_dot_adj_rating[User_dot_adj_rating['movieId']==movie]

    if len(np.unique(User_dot_adj_rating1['userId'])) >= 2:

        User_dot_adj_rating1['wt_rating'] = User_dot_adj_rating1['dot']*User_dot_adj_rating1['rating_adjusted']

        User_dot_adj_rating1['dot_abs'] = User_dot_adj_rating1['dot'].abs()
        User_dot_adj_rating1 = User_dot_adj_rating1.groupby(['userId1'], as_index = False, sort = False).sum()[['userId1','wt_rating','dot_abs']]
        User_dot_adj_rating1['Rating'] = (User_dot_adj_rating1['wt_rating']/User_dot_adj_rating1['dot_abs'])+user1_data['rating']
        User_dot_adj_rating1['movieId'] = movie
        User_dot_adj_rating1 = User_dot_adj_rating1.drop(['wt_rating', 'dot_abs'], axis=1)

        User_dot_adj_rating_all = pd.concat([User_dot_adj_rating_all, User_dot_adj_rating1], ignore_index=True)

        User_dot_adj_rating_all = User_dot_adj_rating_all.sort_values(['Rating'], ascending=False)
    return User_dot_adj_rating_all

In [50]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield (i, i+n)

frame = pd.read_csv('data/parsed_qualifying.txt')
length = len(frame)

list(chunks(1000, 10))

TypeError: object of type 'int' has no len()

In [43]:
from tqdm import tqdm
prediction = pd.DataFrame()
i = 0
with open('data/parsed_qualifying.txt', 'r') as f:
    for j, l in enumerate(tqdm(f.readlines())):
        row = l.strip().split(',')
        # print(row)
        user_movie_rating = get_movie_rating(movie_id=int(row[0]), user_id=int(row[1]))
        # print(user_movie_rating)
        pd.concat([prediction, user_movie_rating])
        i = i + 1
        if i > 100:
            break
prediction
        

  0%|          | 92/2817131 [01:32<782:34:41,  1.00s/it]


KeyError: 'movieId'

""


In [63]:
get_movie_rating(movie_id=int(row[0]), user_id=int(row[1]))

KeyError: 'Rating'